In [1]:
import pandas as pd 
import numpy as np
import sklearn

In [24]:
interaction_2020_path = "../uwwi_datasets/uwwi_dataset_interactions/uwwi_dataset_interactions_2020.csv"
interaction_2021_path = "../uwwi_datasets/uwwi_dataset_interactions/uwwi_dataset_interactions_2021.csv"
interaction_2022_path = "../uwwi_datasets/uwwi_dataset_interactions/uwwi_dataset_interactions_2022.csv"
agencies_path = "../uwwi_datasets/uwwi_dataset_agencies.csv"
clients_path = "../uwwi_datasets/uwwi_dataset_clients.csv"
services_path = "../uwwi_datasets/uwwi_dataset_services.csv"
sites_path = "../uwwi_datasets/uwwi_dataset_sites.csv"
census_svi_path = "../external_datasets/Wisconsin_CENSUS_SVI.csv"
county_svi_path = "../external_datasets/Wisconsin_COUNTY_SVI.csv"

In [7]:
import chardet
def get_encoding(file_path):

    with open(file_path,'rb') as f:
            data = f.read(10000)
    return chardet.detect(data).get("encoding")

In [77]:
file_path = clients_path
df = pd.read_csv(file_path, sep=',')
df.head(5)

C:\Users\Siddiquee\AppData\Local\Temp\ipykernel_5720\3683218573.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=',')


,Client_Id,Client_CreateStamp,Client_EditStamp,ClientOption_PreferredLanguageOfCaller,ClientSystem_PreferredLanguageOther,ClientAddressus_ClientAddressus_city,ClientAddressus_ClientAddressus_county,ClientAddressus_ClientAddressus_state,ClientAddressus_ClientAddressus_zip,ClientCustom_AgeOfPersonNeedingAssistance,ClientCustom_EthnicityOther,ClientOption_GenderOptionId,ClientOption_VeteranStatusOptionlist
0,1,2017-04-07T15:34:04.872169-05:00,2022-04-23T00:53:31.812322-05:00,[],NaN,FOND DU LAC,FOND DU LAC,WI,54935.0,NaN,NaN,[],[]
1,2,2017-04-10T08:08:10.404723-05:00,2022-04-23T00:53:33.172375-05:00,[],NaN,MADISON,DANE,WI,53705.0,NaN,NaN,[],[]
2,3,2017-04-10T08:21:14.98351-05:00,2022-02-28T19:01:21.95251-06:00,[],NaN,NaN,NaN,NaN,54115.0,NaN,NaN,[],[]
3,4,2017-04-10T08:25:16.115921-05:00,2022-04-23T00:53:33.445243-05:00,[],NaN,MENASHA,WINNEBAGO,WI,54952.0,NaN,NaN,[],[]
4,5,2017-04-10T08:42:59.367602-05:00,2022-04-23T00:53:33.678873-05:00,[],NaN,NEENAH,WINNEBAGO,WI,54956.0,NaN,NaN,[],[]


In [86]:
df = df[df.ClientAddressus_ClientAddressus_state.notna()]
counties = df.ClientAddressus_ClientAddressus_state.str.lower().unique().tolist()
counties = list(set([name.strip() for name in counties]))
counties.sort()
with open('./county_names_clients.txt', 'w') as file:
    for line in counties:
        file.write(f"{line}\n")
# df.ClientAddressus_ClientAddressus_county[df.ClientAddressus_ClientAddressus_state.isna()]

In [93]:
df.shape[0] - df[df.ClientAddressus_ClientAddressus_state.str.lower() == 'wi'].shape[0]

19952

In [94]:
df = df[df.ClientAddressus_ClientAddressus_state.str.lower() == 'wi']

In [96]:
df.ClientAddressus_ClientAddressus_county = df.ClientAddressus_ClientAddressus_county.astype(str)
names = df.ClientAddressus_ClientAddressus_county.str.lower().unique().tolist()
names = list(set([name.strip() for name in names]))
names.sort()
with open('./county_names.txt', 'w') as file:
    for line in names:
        file.write(f"{line}\n")

C:\Users\Siddiquee\AppData\Local\Temp\ipykernel_5720\3704785758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ClientAddressus_ClientAddressus_county = df.ClientAddressus_ClientAddressus_county.astype(str)


In [23]:
df.ClientAddressus_ClientAddressus_zip[df.ClientAddressus_ClientAddressus_zip.isna()].shape

(68132,)

In [76]:
official_names = df.COUNTY.str.lower().unique().tolist()
official_names = list(set([name.strip() for name in official_names]))
official_names.sort()
with open('./official_county_names.txt', 'w') as file:
    for line in official_names:
        file.write(f"{line}\n")

# ADI Scores

In [141]:
SOCIOECONOMIC_FILE = '../external_datasets/socioeconomic_dataset/socioeconomic_scores_zipcode.csv'
df_socioec_scores = pd.read_csv(SOCIOECONOMIC_FILE)
df_socioec_scores.head(5)

,ZIP_4,GISJOIN,FIPS,ADI_NATRANK,ADI_STATERANK,TYPE
0,530010064,G55011700110002,5.511701e+11,47,4,
1,530010061,G55011700110002,5.511701e+11,47,4,
2,530010063,G55011700110002,5.511701e+11,47,4,
3,530010049,G55011700112001,5.511701e+11,45,3,
4,530010050,G55011700112001,5.511701e+11,45,3,


In [99]:
def clean_adi(df_original):
    
    # Let's create the cleaned dataset
    df_clean = df_original.copy()
    
    # Extract only 5 digits from zipcode
    df_clean['ZIP'] = df_clean['ZIP_4'].astype(str).str[:5]
    
    # Remove zipcodes that don't have a valid decile
    df_clean = df_clean.loc[(df_clean.ADI_STATERANK!='GQ') &
                                         (df_clean.ADI_STATERANK!='PH') &
                                          (df_clean.ADI_STATERANK!='GQ-PH') &
                                        (~df_clean.ADI_STATERANK.isna())
                                         ]
    # Transform state rank to integer
    df_clean['ADI_STATERANK'] = df_clean['ADI_STATERANK'].astype(int)
    
    # IMPORTANT: Assign each zipcode the average rank from their block neighborhoods 
    df_clean = df_clean[['ZIP','ADI_STATERANK']].groupby(['ZIP']).\
                                                                agg({'ADI_STATERANK':'mean'}).reset_index()
    
    
    return df_clean[['ZIP','ADI_STATERANK']]

In [101]:
df = clean_adi(df_socioec_scores)
df.head(5)

,ZIP,ADI_STATERANK
0,53001,3.680292
1,53002,4.310638
2,53003,2.000000
3,53004,2.960118
4,53005,1.497231


In [140]:
# df[['ZIP, COUNT']].groupby(['ZIP']).count()
# ['ADI_STATERANK']
counting = df.groupby(["ZIP"]).count().reset_index()
counting[counting.ADI_STATERANK > 1]

,ZIP,ADI_STATERANK


In [144]:
df_socioec_scores.ADI_NATRANK.unique()

array(['47', '45', '59', '44', '57', '26', '43', '39', '33', '41', '54',
       '19', '24', '29', '25', '36', '30', '13', '31', '38', '35', '23',
       '27', '22', '14', '46', '42', '53', '58', '61', '68', '64', '20',
       '37', '34', '21', '18', '32', '52', '67', '56', '78', '50', '63',
       '62', '69', '16', '51', '17', '9', '66', '40', '48', '49', '72',
       '74', '11', 'GQ', '55', '6', '12', '71', '81', '60', '28', '79',
       '76', '75', '8', '7', '77', '65', '10', '5', '87', '92', '93',
       '70', '83', '89', '96', '88', '86', '94', '73', '15', '82', '85',
       'GQ-PH', '97', '91', '84', '80', '100', '90', 'PH', '98', '95',
       '99', '4', nan, '3'], dtype=object)